In [2]:
import numpy as np

np.random.seed(42)  # For reproducibility

# Define the discrete state space with numeric values
states = [1, 7, -4, 3.333, 999.1]

# For simplicity, let’s assume we are modeling a process with three time steps: T0, T1, T2
# and the transitions from T0 -> T1 depend only on X0,
# while the transitions from T1 -> T2 depend on (X0, X1).
# This can be easily extended to more steps.

# Transitions from T0 to T1:
# P(X1 = s_j | X0 = s_i)
transitions_T0_T1 = {
    1:    {1:0.1, 7:0.3, -4:0.2, 3.333:0.3, 999.1:0.1},
    7:    {1:0.05,7:0.05,-4:0.2, 3.333:0.3, 999.1:0.4},
    -4:   {1:0.4, 7:0.1,-4:0.1, 3.333:0.3, 999.1:0.1},
    3.333:{1:0.2, 7:0.2,-4:0.2, 3.333:0.2, 999.1:0.2},
    999.1:{1:0.0, 7:0.5,-4:0.1, 3.333:0.3, 999.1:0.1}
}

# Transitions from T1 to T2 depend on both X0 and X1:
# P(X2 = s_k | X0 = s_i, X1 = s_j)
transitions_T1_T2 = {
    (1, 1):      {1:0.3, 7:0.1, -4:0.2, 3.333:0.3, 999.1:0.1},
    (1, 7):      {1:0.05,7:0.05,-4:0.3, 3.333:0.3, 999.1:0.3},
    (1, -4):     {1:0.1, 7:0.4, -4:0.1, 3.333:0.2, 999.1:0.2},
    (1, 3.333):  {1:0.25,7:0.25,-4:0.25,3.333:0.2, 999.1:0.05},
    (1, 999.1):  {1:0.3, 7:0.1, -4:0.3, 3.333:0.2, 999.1:0.1},

    (7, 1):      {1:0.4, 7:0.1, -4:0.1, 3.333:0.3, 999.1:0.1},
    (7, 7):      {1:0.1, 7:0.1, -4:0.3, 3.333:0.2, 999.1:0.3},
    (7, -4):     {1:0.2, 7:0.2, -4:0.2, 3.333:0.2, 999.1:0.2},
    (7, 3.333):  {1:0.05,7:0.35,-4:0.15,3.333:0.4, 999.1:0.05},
    (7, 999.1):  {1:0.1, 7:0.3, -4:0.2, 3.333:0.1, 999.1:0.3},

    (-4, 1):     {1:0.3, 7:0.3, -4:0.1, 3.333:0.2, 999.1:0.1},
    (-4, 7):     {1:0.2, 7:0.05,-4:0.1, 3.333:0.25,999.1:0.4},
    (-4, -4):    {1:0.05,7:0.05,-4:0.4, 3.333:0.4, 999.1:0.1},
    (-4, 3.333): {1:0.3, 7:0.3, -4:0.1, 3.333:0.1, 999.1:0.2},
    (-4, 999.1): {1:0.1, 7:0.2, -4:0.3, 3.333:0.3, 999.1:0.1},

    (3.333, 1):  {1:0.25,7:0.25,-4:0.25,3.333:0.15,999.1:0.1},
    (3.333, 7):  {1:0.1, 7:0.4, -4:0.05,3.333:0.25,999.1:0.2},
    (3.333, -4): {1:0.2, 7:0.2, -4:0.2, 3.333:0.2, 999.1:0.2},
    (3.333, 3.333):{1:0.1,7:0.3,-4:0.2,3.333:0.3,999.1:0.1},
    (3.333, 999.1):{1:0.05,7:0.05,-4:0.4,3.333:0.4,999.1:0.1},

    (999.1, 1):  {1:0.4, 7:0.2, -4:0.2, 3.333:0.1, 999.1:0.1},
    (999.1, 7):  {1:0.05,7:0.05,-4:0.3, 3.333:0.3, 999.1:0.3},
    (999.1, -4): {1:0.1, 7:0.1, -4:0.1, 3.333:0.4, 999.1:0.3},
    (999.1, 3.333):{1:0.3,7:0.3,-4:0.3,3.333:0.05,999.1:0.05},
    (999.1, 999.1):{1:0.1,7:0.1,-4:0.1,3.333:0.3,999.1:0.4}
}

def sample_from_distribution(distribution):
    """Given a dict {state:prob}, randomly sample a state according to these probabilities."""
    st_list = list(distribution.keys())
    probs = [distribution[s] for s in st_list]
    return np.random.choice(st_list, p=probs)

def simulate_3_step_process(X0, transitions_T0_T1, transitions_T1_T2):
    """
    Simulate one realization of a 3-step process:
    T0: given by X0
    T1: depends only on X0
    T2: depends on (X0, X1)
    """
    # Sample X1
    dist_T1 = transitions_T0_T1[X0]
    X1 = sample_from_distribution(dist_T1)

    # Sample X2
    dist_T2 = transitions_T1_T2[(X0, X1)]
    X2 = sample_from_distribution(dist_T2)

    return [X0, X1, X2]

# Example simulation:
X0_initial = 7
trajectory = simulate_3_step_process(X0_initial, transitions_T0_T1, transitions_T1_T2)
print("One sampled trajectory (3-step):", trajectory)

# If we wanted to simulate longer processes with varying lengths, we can build a function for that:
def simulate_longer_process(initial_state, length):
    if length == 1:
        return [initial_state]
    elif length == 2:
        # Just one step transition from T0->T1
        dist = transitions_T0_T1[initial_state]
        return [initial_state, sample_from_distribution(dist)]
    else:
        # length >= 3
        X = [initial_state]
        # Step 1
        X.append(sample_from_distribution(transitions_T0_T1[X[0]]))
        # Steps from T2 onwards:
        for t in range(2, length):
            dist = transitions_T1_T2[(X[t-2], X[t-1])]
            X.append(sample_from_distribution(dist))
        return X

# Simulate two processes of different lengths:
X_short = simulate_longer_process(initial_state=1, length=5)
X_long = simulate_longer_process(initial_state=1, length=8)

print("Short process (length=5):", X_short)
print("Long process (length=8):", X_long)

One sampled trajectory (3-step): [7, 3.333, 999.1]
Short process (length=5): [1, 3.333, -4.0, 1.0, 1.0]
Long process (length=8): [1, 1.0, 3.333, -4.0, 3.333, 1.0, 999.1, 3.333]


In [3]:
import numpy as np

np.random.seed(123)  # for reproducibility

# Define a finite numeric state space
states = [0.5, 10, 20, 1.2345, -100]

# The process:
# - For T0->T1, transitions depend on X0 only.
# - For T1->T2 and onwards, transitions depend on (X_{t-2}, X_{t-1}).

# Transitions from T0 to T1:
# P(X1 = s_j | X0 = s_i)
transitions_single = {
    0.5:   {0.5:0.2, 10:0.1, 20:0.3, 1.2345:0.3, -100:0.1},
    10:    {0.5:0.05,10:0.25,20:0.25,1.2345:0.25,-100:0.2},
    20:    {0.5:0.4, 10:0.1, 20:0.1, 1.2345:0.1, -100:0.3},
    1.2345:{0.5:0.2, 10:0.2, 20:0.2, 1.2345:0.1, -100:0.3},
    -100:  {0.5:0.1, 10:0.4, 20:0.05,1.2345:0.4, -100:0.05}
}

# Transitions from T_(t-1) to T_t (for t>=2):
# P(X_t = s_k | X_{t-2}=s_i, X_{t-1}=s_j)
# We'll define a dictionary that gives a distribution for each pair (s_i, s_j).
# This will be re-used for all t>=2 (time-homogeneous for steps beyond the first, just as an example).
transitions_pair = {
    (0.5, 0.5):     {0.5:0.1, 10:0.3, 20:0.1, 1.2345:0.4, -100:0.1},
    (0.5, 10):       {0.5:0.2, 10:0.2, 20:0.2, 1.2345:0.2, -100:0.2},
    (0.5, 20):       {0.5:0.05,10:0.05,20:0.4, 1.2345:0.4, -100:0.1},
    (0.5, 1.2345):   {0.5:0.3, 10:0.1, 20:0.1, 1.2345:0.4, -100:0.1},
    (0.5, -100):     {0.5:0.25,10:0.25,20:0.1, 1.2345:0.1, -100:0.3},

    (10, 0.5):       {0.5:0.3, 10:0.3, 20:0.1, 1.2345:0.2, -100:0.1},
    (10, 10):        {0.5:0.1, 10:0.1, 20:0.3, 1.2345:0.3, -100:0.2},
    (10, 20):        {0.5:0.2, 10:0.05,20:0.05,1.2345:0.4, -100:0.3},
    (10, 1.2345):    {0.5:0.25,10:0.15,20:0.05,1.2345:0.5, -100:0.05},
    (10, -100):      {0.5:0.1, 10:0.4, 20:0.1, 1.2345:0.3, -100:0.1},

    (20, 0.5):       {0.5:0.4, 10:0.05,20:0.05,1.2345:0.25,-100:0.25},
    (20, 10):        {0.5:0.05,10:0.2, 20:0.2, 1.2345:0.3, -100:0.25},
    (20, 20):        {0.5:0.1, 10:0.3, 20:0.2, 1.2345:0.3, -100:0.1},
    (20, 1.2345):    {0.5:0.2, 10:0.3, 20:0.2, 1.2345:0.2, -100:0.1},
    (20, -100):      {0.5:0.1, 10:0.05,20:0.05,1.2345:0.3, -100:0.5},

    (1.2345, 0.5):   {0.5:0.2, 10:0.2, 20:0.2, 1.2345:0.2, -100:0.2},
    (1.2345, 10):    {0.5:0.3, 10:0.1, 20:0.1, 1.2345:0.3, -100:0.2},
    (1.2345, 20):    {0.5:0.1, 10:0.1, 20:0.4, 1.2345:0.3, -100:0.1},
    (1.2345, 1.2345):{0.5:0.05,10:0.05,20:0.3, 1.2345:0.5, -100:0.1},
    (1.2345, -100):  {0.5:0.15,10:0.4, 20:0.05,1.2345:0.3, -100:0.1},

    (-100, 0.5):     {0.5:0.05,10:0.15,20:0.3, 1.2345:0.4, -100:0.1},
    (-100, 10):      {0.5:0.2, 10:0.05,20:0.15,1.2345:0.3, -100:0.3},
    (-100, 20):      {0.5:0.3, 10:0.2, 20:0.1, 1.2345:0.2, -100:0.2},
    (-100, 1.2345):  {0.5:0.25,10:0.2, 20:0.1, 1.2345:0.25,-100:0.2},
    (-100, -100):    {0.5:0.1, 10:0.1, 20:0.3, 1.2345:0.1, -100:0.4}
}

def sample_from_distribution(distribution):
    """Given a dict {state: probability}, sample a state according to these probabilities."""
    st_list = list(distribution.keys())
    probs = [distribution[s] for s in st_list]
    return np.random.choice(st_list, p=probs)

def simulate_process(initial_state, length, transitions_single, transitions_pair):
    """
    Simulate a discrete-time, discrete-state stochastic process with:
    - A finite numeric state space.
    - For length=1: Just return the initial state.
    - For length>=2: The second state is chosen using transitions_single based on X0.
    - For length>=3: Subsequent states are chosen from transitions_pair based on (X_{t-2}, X_{t-1}).

    Parameters:
    initial_state: The starting state X0
    length: number of steps in the process
    transitions_single: dict for transitions from T0->T1
    transitions_pair: dict for transitions from T_(t-1)->T_t (t>=2), keyed by (X_{t-2}, X_{t-1})

    Returns:
    A list of states [X0, X1, X2, ... X_{length-1}]
    """
    if length <= 0:
        return []
    if length == 1:
        return [initial_state]

    # length >= 2
    # Determine X1
    X = [initial_state]
    dist_for_X1 = transitions_single[X[0]]
    X.append(sample_from_distribution(dist_for_X1))

    # length >= 3
    for t in range(2, length):
        pair = (X[t-2], X[t-1])
        dist_for_next = transitions_pair[pair]
        X.append(sample_from_distribution(dist_for_next))

    return X

# Example simulations:
X_process_1 = simulate_process(initial_state=0.5, length=5, 
                               transitions_single=transitions_single, 
                               transitions_pair=transitions_pair)
X_process_2 = simulate_process(initial_state=10, length=8, 
                               transitions_single=transitions_single, 
                               transitions_pair=transitions_pair)

print("5-step process starting from 0.5:", X_process_1)
print("8-step process starting from 10:", X_process_2)


5-step process starting from 0.5: [0.5, 1.2345, 0.5, 10.0, 20.0]
8-step process starting from 10: [10, 1.2345, 20.0, -100.0, -100.0, 20.0, 10.0, 20.0]
